# Statistics from label images

In [1]:
#Import required libraries
import os
import python_scripts.morphology as morph

## Provide user defined variables

- Use **absolute pathing** for main directory
- Subdirectories will be automatically identified and scanned
- Graphs and tables will be stored inside a Results folder in the main directory

- The properties variable is a list of properties to calculate for each region. Defaults to ['label', 'area', 'eccentricity',  'perimeter', 'equivalent_diameter_area', 'axis_major_length', 'axis_minor_length', 'area_filled']


### User defined variables

In [2]:
# Directory with folders of sampling folders with GT and Prediction folder pairs inside
# Expected file arrangement: given_folder contains the OG and downsampling_2 folders, then each of those contains GT and Prediction folders
directory = '/Users/marianaferreira/Library/CloudStorage/OneDrive-igc.gulbenkian.pt/OCB/Downsampling/Analysis_folders_QC/Saureus'

# User defined properties to calculate using region.props
properties =['label', 'area', 'eccentricity',  'perimeter', 'equivalent_diameter_area','axis_major_length', 'axis_minor_length', 'area_filled']

#Pixel size in calibrated units use spacing = (1,1) for images with unknown calibration
#spacing = (0.0322421, 0.0322421) #Saureus
spacing = (1, 1)

# Dictionary identifying sampling multipliers according to folder names
folder_sampling_dict = {'upsampling_16': 16, 'upsampling_8': 8, 'upsampling_4': 4,'upsampling_2': 2, 'OG': 1, 'downsampling_2': 1/2, 'downsampling_4': 1/4, 'downsampling_8': 1/8, 'downsampling_16': 1/16}

# List of possible columns to plot in graphs
cols_to_plot = ['Circularity', 'Roundness', 'Filledness', 'Pixel_Coverage_Percent', 'norm_area', 'norm_perimeter', 'IoU', 'f1_score']

In [3]:
# Create folder to store results if it doesn't exist
res_dir = os.path.join(directory, 'Results')

if not os.path.exists(res_dir):
    os.mkdir(res_dir)

### Statistics

In [4]:
#Calculate object properties statistics
obj_props_df = morph.object_props(directory, properties, spacing, folder_sampling_dict)

# Save object properties statistics in csv file
obj_props_df.to_csv(os.path.join(res_dir, directory.split('/')[-1] +'_region_props.csv'))    

# Get list of parent folders from the object properties dataframe
parent_folder_dict = morph.parent_folder_dict(obj_props_df)

# Calculate prediction statistics
pred_stats_df = morph.prediction_statistics(parent_folder_dict, directory)

# Save prediction statistics in csv file
pred_stats_df.to_csv(os.path.join(res_dir, directory.split('/')[-1] +'_prediction_stats.csv'))  



In [ ]:

summary_df, IoU_per_obj_df = morph.per_object_statistics(directory, res_dir, obj_props_df)

# Save summary statistics in csv file
summary_df.to_csv(os.path.join(res_dir, directory.split('/')[-1] +'_summary_stats.csv'))    

# Save IoU per object statistics in csv file
IoU_per_obj_df.to_csv(os.path.join(res_dir, directory.split('/')[-1] +'_IoU_per_obj_stats.csv'))


### Graph

In [ ]:
for col in cols_to_plot:
    if col in obj_props_df.columns:
        if col == 'Pixel_Coverage_Percent':
            morph.generate_basic_plot(res_dir, obj_props_df, folder_sampling_dict, col, 'box_no_outliers', True)
        else:    
            morph.generate_basic_plot(res_dir, obj_props_df, folder_sampling_dict, col, 'box_no_outliers', False)

for col in ['IoU', 'f1_score']:
    morph.generate_basic_plot(res_dir, pred_stats_df, folder_sampling_dict, col, 'strip', False, None)

for col in ['Mean_IoU', 'Mean_f1_score','true_positives_count', 'false_positives_count', 'false_negatives_count', 'Accuracy', 'Sensitivity']:
    morph.generate_basic_plot(res_dir, summary_df, folder_sampling_dict, col, 'strip', False, None)
    
